In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"
import argparse
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import random
# from create_embeddings import create_embeddings
import numpy as np
from sklearn.model_selection import train_test_split
from Create_simple_train_tsv import get_gene_fam_per_gene, create_similar_genes

In [43]:
def create_segment_2(df: pd.DataFrame):
    df.seq_x = df.seq_x.astype("string")
    df.seq_y = df.seq_y.astype("string")

    halfa = df.sample(frac=0.5,random_state=42)
    halfb = df[~df.index.isin(halfa.index)]
    halfa = halfa.add_suffix("a")
    halfb = halfb.add_suffix("b")
    halfb.reset_index(inplace=True)
    halfb.rename(columns={"index":'segment_id_x'}, inplace=True)
    halfa.reset_index(inplace=True)
    halfa.rename(columns={"index":'segment_id_y'}, inplace=True)
    df = pd.concat([halfa, halfb], axis=1)
    # print(df.head())
    # print(df.columns)
    
    
    df1 = df.sample(frac=0.5,random_state=42)
    df2 = df[~df.index.isin(halfa.index)]
    df1.reset_index(inplace=True)
    df2.reset_index(inplace=True)
    
    
    df1["seq_x"] = df1.seq_xa + df1.seq_xb
    df1["seq_y"] = df1.seq_ya + df1.seq_yb

    df2["seq_x"] = df2.seq_xa + df2.seq_xb
    df2["seq_y"] = df2.seq_yb + df2.seq_ya
    df = pd.concat([df1, df2]).reset_index()    
    df.drop(columns=['seq_xa', 'seq_ya', 'seq_xb', 'seq_yb', "index"], inplace=True)
    return  df


def create_negative_segment(df: pd.DataFrame):
    """Generates negative samples (not similar)
    Negatives samples are created by randomly selecting another gene from a different faimly.
    No duplicates should appear

    Args:
        df (pd.DataFrame): output from create_similar_genes

    Returns:
        pd.Dataframe: df with negative samples
    """
    df_selection = df.copy()
    # print(df.head())
    groups = []
    for (fama, famb), group in df.groupby(["familya", "familyb"]):
        shuffeld = group.copy()
        temp = df_selection.query('(familya != @fama) & (familyb != @famb)').sample(n=shuffeld.shape[0], random_state=42)
        df_selection = df_selection[~df_selection.index.isin(temp.index)]
        shuffeld[["segment_id_y", "gene_ya",  "gene_yb", "seq_y"]] = temp[["segment_id_y", "gene_ya", "gene_yb", "seq_y"]].values
        groups.append(shuffeld)
    return pd.concat(groups)

def create_train_test_val(df):
    x_index, x_val_index, _, _ = train_test_split(df.index, df.similar.astype(int).values, test_size=0.30, random_state=42)
    df_val = df.iloc[x_val_index]
    
    df = df.iloc[x_index].reset_index(drop=True)
    x_train_index, x_test_index, _, _ = train_test_split(df.index, df.similar.astype(int).values, test_size=0.20, random_state=42)
    return df.iloc[x_train_index], df.iloc[x_test_index], df_val    


In [3]:
trans_p="/home/jong505/thesis/iadh/data/"

refseqs = [f"{trans_p}annotation.all_transcripts.aar.csv.gz", f"{trans_p}/annotation.all_transcripts.ath.csv.gz", f"{trans_p}/annotation.all_transcripts.bol.csv.gz",
           f"{trans_p}annotation.all_transcripts.chi.csv.gz", f"{trans_p}/annotation.all_transcripts.cpa.csv.gz", f"{trans_p}/annotation.all_transcripts.tha.csv.gz"]
gene_fam = Path(f"{trans_p}gene_fam_parsed.tsv")
refseqs = [Path(f) for f in refseqs]
output_prefix = Path("data/aar_ath_bol_chi_cpa_tha/medium_2g_500")
output_prefix_raw = Path("data/aar_ath_bol_chi_cpa_tha/medium_2g_500")
sample_size = 500

if not output_prefix.parent.is_dir():
    raise ValueError(f"Output folder {output_prefix.parent} doesn't exist")

df = get_gene_fam_per_gene(gene_fam, refseqs)

starting with /home/jong505/thesis/iadh/data/annotation.all_transcripts.aar.csv.gz
starting with /home/jong505/thesis/iadh/data/annotation.all_transcripts.ath.csv.gz


/home/jong505/thesis/model/Create_simple_train_tsv.py:37: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  dfn = pd.read_csv(path, compression='gzip', sep='\t', header=0, skiprows=8)


starting with /home/jong505/thesis/iadh/data/annotation.all_transcripts.bol.csv.gz
starting with /home/jong505/thesis/iadh/data/annotation.all_transcripts.chi.csv.gz
starting with /home/jong505/thesis/iadh/data/annotation.all_transcripts.cpa.csv.gz
starting with /home/jong505/thesis/iadh/data/annotation.all_transcripts.tha.csv.gz


In [21]:
df1 = create_similar_genes(df)

In [39]:
test = create_segment_2(df1)
print(test.shape)
test.sort_values("segment_id_y")

(1495, 11)


,level_0,segment_id_y,familya,gene_xa,gene_ya,segment_id_x,familyb,gene_xb,gene_yb,seq_x,seq_y
314,314,6,HOM05D000001,Aa31LG8G3100,CARHR111320,5327,HOM05D000873,Aa31LG2G22080,Cpa.g.sc27.104,ATGATTCTACGAAACTCAGTAAGATCTTCAAGAAACACTAAATCAC...,ATGGCTCCTACAGGGAACGGGCCATTGATACCAATTCTAGGTTTCC...
708,708,12,HOM05D000001,Aa31LG8G3100,CARHR141320,2455,HOM05D000016,CARHR140780,THA.LOC104804254,ATGATTCTACGAAACTCAGTAAGATCTTCAAGAAACACTAAATCAC...,ATGTCCCAAGAGTCACGAGTCACGACCCTCGAGTTAAGTCAGAGAT...
101,101,14,HOM05D000001,Aa31LG8G3100,Cpa.g.sc211.28,754,HOM05D000001,Cpa.g.sc5.192,THA.LOC104799425,ATGATTCTACGAAACTCAGTAAGATCTTCAAGAAACACTAAATCAC...,ATGGTTTCTGCGGGAGTTTGTCCTACTCGTTATGCGCTCGGTAGTG...
1442,1442,15,HOM05D000001,Aa31LG8G3100,Cpa.g.sc1.31,935,HOM05D000003,Aa31LG2G17440,CARHR219390,ATGATTCTACGAAACTCAGTAAGATCTTCAAGAAACACTAAATCAC...,ATGTACCTGGTATCGTCATGTAAATCAGACAAGCACCTCCACCAAG...
82,82,19,HOM05D000001,Aa31LG8G3100,Cpa.g.sc18.113,3685,HOM05D000111,AT2G20170,BolC4t22155H,ATGATTCTACGAAACTCAGTAAGATCTTCAAGAAACACTAAATCAC...,ATGAGCATAACCCTCAGTTTCAGAACCCTAATAACAAAGAAGCAAA...
...,...,...,...,...,...,...,...,...,...,...,...
1472,1472,5958,HOM05D006622,Aa31LG4G7650,Cpa.g.sc64.93,3678,HOM05D000111,Aa31LG6G6580,BolC4t22155H,ATGTTCGATCAATTGGTATTGTTTGGTGGTACTTGGCACTGGGCGT...,ATGGCAAGAACAAGCAATACTCAGAGCAGCAGCAAGGCTGGTGCTT...
461,461,5970,HOM05D007593,AT1G10417,CARHR010560,4808,HOM05D000370,AT5G66600,THA.LOC104798362,ATGGAGATGCTCAAGTTGTCAAAATTCAAATTGCAGCTTCAGTCAA...,ATGGAGCTGCTCAAGTTGTCAAAATTCAAATTGCAGCTTCAGTCAA...
356,356,5975,HOM05D009162,AT1G65486,BolC2t08280H,5650,HOM05D001373,Aa31LG8G14140,AT2G16990,ATGACGAAGAAAAAAATGGGTTTGATGTCACCAAATATCGCTGCAT...,ATGGGTCCAAAAACATCACATAAGGTTGCTCTTTTGTTCTCTCTTC...
168,168,5976,HOM05D009783,AT5G64900,BolC3t18389H,1100,HOM05D000003,Aa31sc407G40,Cpa.g.sc35.51,ATGGAGAAATCAGATAGACGAAGCGAAGAAAGTCACCTATGGATTC...,ATGGAAAAAATTGAGAGACAAACCGAAGAAGCATCTTATCTAGGGC...


In [35]:
print(df.query('family_id == "HOM05D000003" & gene_id =="Aa31sc146G90"').seq.values[0] + df.query('family_id == "HOM05D000040" & gene_id =="Aa31LG1G5240"').seq.values[0])
print(test.iloc[0].seq_x)
print("-"*20)
print(df.query('family_id == "HOM05D000003" & gene_id =="BolC3t20183H"').seq.values[0] + df.query('family_id == "HOM05D000040" & gene_id =="AT1G31450"').seq.values[0])
print(test.iloc[0].seq_y)
print("-"*20)

ATGCTTAGAACACATGTACATGAAGAAGACTTGGAATGGGTTAAGCAGGTAATTCCGGGTATTGATTCATCAAATGACTCCTACGTGTGGAATTATTCAAAGGATGGAGAATACTCTGTAAGGTCTGGAACATGGGTATACAAAAATGAAGGACAAAATCAACGCAGAACAGAGAACCCGAACCTACAACCTTTGTACAAATCAATATGGGAATCAAGAACAATACCTAAGATCAAAAACTTCATGTGGAGAGTGGTCTCAAATTGCTTAGCAGTAGGAGGGAATATGAAAAGGAAGAAGCTAACAAAGGAAGGTTTATGCCCGTTTTGTAAAACACAGGAAGAAACGGTGAATCACCTGTTGTTCACATGTCCTTTTGCAAGGTTAGTCTGGGCGGTGAGCGGAATACCGGGAATACAGAGCTCTCCCTTTAGTGAGTCTGTGTACTCCAACATAGCACGTTTGCTATGGCAACAAGCACCTGAAAAGGAAATAAGAGAACTGTTGTTAAAAGGACCCTGGATACTATGGGGGATTTGGAAATCAAGGAATGATGCAGTATTCAAAGGACTGCCACCATCAGAGATGAAGATCAGAACCAAAGCAGATATCGGAGCTGAGGAATGGAGAAGTACTGAACAAGAAAAGGCACAAGAGCAAACCAGAATGCAGGGTCCGAGAAATCGTGACCCGATCCGAGTAGATCAGGCTCGTAGAGCATGGAGACCACCGAGAGAAGGTACCTGGAAATGTAATATAGACGGTGCATGTGAGGATGGAGGAGCGAGTGCAGTAGGATGGATTCTCAGAGATTGGAGAGGCCAAACGAGAATGTGGGGAGGCAAAAAGCTTGAAATCACAAAACTGATCCAGCATCAAACAGTCCTATCGGTTGAGGCAGAGGCACTAACCTGGGCTATACTACAGATCAAGGATCTAGGTCTCGGGAGGGTTGAAATAGAATCTGACTGTGGCGATCTGGTTAAAGGACTCACAATTG

In [44]:
df_negatives = create_negative_segment(test)

In [42]:
test.query('gene_ya == "THA.LOC104809906" and gene_yb == "BolC7t45641H"')

,level_0,segment_id_y,familya,gene_xa,gene_ya,segment_id_x,familyb,gene_xb,gene_yb,seq_x,seq_y
56,56,2481,HOM05D000017,Aa31LG11G4650,THA.LOC104809906,2969,HOM05D000048,AT3G23560,BolC7t45641H,ATGACTGATTTTGATGATGAAGCTGAAGATAGAGACCAGAATCTCG...,ATGAGGAAAATGATTGTAGTTGATTGTACATTCTTGACTGGCAAGT...


In [51]:
print(df_negatives.query('segment_id_x == 2969').seq_y.values[0])
print(test.query('segment_id_x == 2969').seq_y.values[0])

ATGCTACTTAGAAACATAGATCCTAAAGAGGGTTTGTGCAATGGTACAAGACTTCAGGTTACGAAATTAGCTAACCATGTCACTGAAGCAGAGATCATAACAACAGAAGATGATAAGGTTAAAAAGGTCATGATTCCTAGGATATTTCTTTCTCCATTGGATACAAAGTTTCCTTTCCAGATGAGGCGTCAGCAATTTCCCGTCGCATTAGCATTCATGATGACAATCAACAAAAGCCAAGGTCAAACTTTATCAAGAGTCGGACTCTATCTTCCCCGTCCAGTTTTCTCACACGGTCAGTTGTATGTGGCTCTTTTACGGGTTAAGTCAAAAGATGGTTTGAAGATTTTGATATTAGATGATAAAGGTAAATGTCAACAAAGTTTGGTGCATGGAGCAGATGGTGAAGGCAGGGAAAAACAGCGTAGAAAAGAGGAAGATAGTAAGAAAACAAAGGATGAGGAGCTTGAAGATGGATAAATGTCGACGGAACGAGAAGTAGTCTGCGTCACCGGCGCCAGTGGGGGCATCGGCTCGTGGCTAGTCCATCTACTCCTGCACCGCGGCTACTCCGTTCACGCAACCGTGAAAAACCTCCAGGACGAGAAAGAGACAAAACATCTGGAAGCTCTAGAAGGTGCAGCCACGCGCCTCCATTTATTCGAGATGGATCTCCTCCAATACGACACCGTTTTAACCGCCGTCAACGGATGTTCCGGCGTATTCCATCTCGCATCGCCTTGTATCGTCGACGAAGTCCAAGATCCTCAGAAACAACTACTTGACCCGGCGGTTAAAGGGACACTAAATGTACTAACGGCGGCGAAAGAAGCCGGCGTTAAAAGAGTTGTTGTAACGTCTTCGATATCGGCGATAACTCCAAGTCCTAACTGGCCTGTCGATAAGATCAAGAATGAGGAATGTTGGGCTGATCAAGACTACTGTAACCAACATAAATTGTGGTATCCACTGTCGAAGATACTTGCTGAGAAAGCAGCTT

In [ ]:
df_negatives.insert(loc=7, column="similar", value=False)
test.insert(loc=7, column="similar", value=True)
df = pd.concat([df, df_negatives]).reset_index(drop=True)
train, test, val = create_train_test_val(df)

(1429, 9)

Maybe the segment_id are getting duplicated when concatenating the negative and normal matrix?